In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install necessary libraries
!pip install --upgrade pip
!pip install --upgrade --force-reinstall numpy==2.0.2
!pip install nvidia-cufft-cu12
!pip install torch torchvision albumentations scikit-image pytorch-lightning seaborn captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
captum 0.8.0 requires numpy<2.0, but you have numpy 2.0.2 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux201

This code is partially based on This code is partially based on https://github.com/tchanda90/Derma-XAI.

In [ ]:
from albumentations.pytorch import ToTensorV2
from captum.attr import LayerAttribution, LayerGradCam
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from skimage import io
from sklearn import metrics
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import WeightedRandomSampler
from torchmetrics import Accuracy, AUROC, Recall, Specificity
from torchvision import models
from tqdm.auto import tqdm

import albumentations
import cv2
import json
import numpy as np
import os
import pandas as pd
import pytorch_lightning as pl
import random
import torch
import matplotlib.pyplot as plt
import pickle

# Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cv2.setNumThreads(1)
os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
BASE_DIR = "/content/drive/MyDrive/DATABase/"
IMG_DIR = os.path.join(BASE_DIR, "HAM10000_images")
METADATA_FILE = os.path.join(BASE_DIR, "metadata_ground_truth_with_splits.csv")

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
NUM_EPOCHS = 25
LEARNING_RATE = 0.0001
BATCH_SIZE = 128
IMAGE_SIZE = 224

SAVE_ATTENTION_PLOTS = False
WEIGHTED_SAMPLING= True
ATTENTIOM_WEIGHT = 10
CHAR_WEIGHT = 1
DROPOUT = 0.4

DX_CLASS_LABEL = ['benign_malignant']

SEED = 42

seed_everything(SEED)

# Dataset

In [ ]:
class MelanomaDataset(Dataset):
    def __init__(self, root_dir, metadata, index=None, transform=None):
        self.root_dir = root_dir
        self.metadata = metadata
        if index is not None:
            self.metadata = self.metadata.loc[index]
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, index):
        image_name = self.metadata.iloc[index]['image_id']
        img_path = os.path.join(self.root_dir, image_name + '.jpg')

        image = io.imread(img_path)
        y_dx = torch.tensor(self.metadata.iloc[index][DX_CLASS_LABEL]).float()

        if self.transform:
            image = self.transform(image=image)['image']

        return image, (y_dx, image_name)

In [ ]:
def get_transforms(image_size, full=False):
    if full:
        transforms_train = albumentations.Compose([
            albumentations.Transpose(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.ColorJitter(p=0.5),
            albumentations.OneOf([
                albumentations.MotionBlur(blur_limit=5),
                albumentations.MedianBlur(blur_limit=5),
                albumentations.GaussianBlur(blur_limit=(3, 5)),
                albumentations.GaussNoise(var_limit=(5.0, 30.0)),
            ], p=0.7),
            albumentations.OneOf([
                albumentations.OpticalDistortion(distort_limit=1.0),
                albumentations.GridDistortion(num_steps=5, distort_limit=1.),
                albumentations.ElasticTransform(alpha=3),
            ], p=0.7),
            albumentations.CLAHE(clip_limit=4.0, p=0.7),
            albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            albumentations.CoarseDropout(max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), max_holes=1, p=0.3),
            albumentations.Resize(image_size, image_size),
            albumentations.Normalize(),
            ToTensorV2()
        ])
    else:
        transforms_train = albumentations.Compose([
            albumentations.Transpose(p=0.2),
            albumentations.VerticalFlip(p=0.2),
            albumentations.HorizontalFlip(p=0.2),
            albumentations.ColorJitter(p=0.5),
            albumentations.CLAHE(clip_limit=4.0, p=0.7),
            albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            albumentations.CoarseDropout(max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), max_holes=1, p=0.3),
            albumentations.Resize(image_size, image_size),
            albumentations.Normalize(),
            ToTensorV2()
        ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize(),
        ToTensorV2()
    ])
    return transforms_train, transforms_val

# Model

In [ ]:
class MelanomaClassifier(pl.LightningModule):
    def __init__(self, backbone_model, hidden_size=64, learning_rate=LEARNING_RATE, metadata_file=METADATA_FILE,
                 data_dir=IMG_DIR, batch_size=BATCH_SIZE, weighted_sampling=WEIGHTED_SAMPLING):

        super().__init__()
        self.backbone_model = backbone_model
        self.metadata_file = metadata_file
        self.data_dir = data_dir
        self.weighted_sampling = weighted_sampling
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.train_set, self.val_set, self.test_set = None, None, None

        self.loss = nn.BCELoss()

        self.labels = DX_CLASS_LABEL
        self.num_classes = len(DX_CLASS_LABEL) # == 1

        self.train_transform, self.test_transform = get_transforms(image_size=IMAGE_SIZE)

        if self.backbone_model == 'resnet18':
          resnet = models.resnet18(pretrained=True)
        elif self.backbone_model == 'resnet34':
          resnet = models.resnet34(pretrained=True)
        elif self.backbone_model == 'resnet50':
          resnet = models.resnet50(pretrained=True)
        elif self.backbone_model == 'resnet101':
          resnet = models.resnet101(pretrained=True)
        resnet.fc = nn.Sequential(
            nn.Dropout(p=DROPOUT),
            nn.Linear(in_features=resnet.fc.in_features, out_features=self.num_classes),
        )

        self.base_model = resnet
        self.sigm = nn.Sigmoid()

        self.accuracy = Accuracy(task='binary', threshold=0.5)
        self.auroc = AUROC(task='binary', average='macro')
        self.sensitivity = Recall(task='binary', threshold=0.5)
        self.specificity = Specificity(task='binary', threshold=0.5)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def forward(self, x):
        output = self.sigm(self.base_model(x))
        return output

    def on_train_start(self):
        self.log("hp/lr", self.learning_rate)
        self.log("hp/batch_size", float(self.batch_size))
        self.log("hp/dropout", DROPOUT)
        self.log("hp/weighted_sampling", float(self.weighted_sampling))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_dx, image_name = y
        output = self(x)
        loss = self.loss(output, y_dx)
        self.log("train/loss", loss, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_dx, image_name = y
        with torch.no_grad():
            output = self(x)
        loss = self.loss(output, y_dx)
        self.log("val/loss", loss, on_epoch=True, on_step=False)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_dx, image_name = y
        with torch.no_grad():
            output = self(x)
        loss = self.loss(output, y_dx)
        self.accuracy(output.flatten(), y_dx.int().flatten())
        self.auroc(output.flatten(), y_dx.int().flatten())
        self.sensitivity(output.flatten(), y_dx.int().flatten())
        self.specificity(output.flatten(), y_dx.int().flatten())

    def on_test_epoch_end(self) -> None:
        self.log("test/bal_acc", self.accuracy.compute())
        self.log("test/auroc", self.auroc.compute())
        self.log("test/sensitivity", self.sensitivity.compute())
        self.log("test/specificity", self.specificity.compute())

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        x, y = batch
        y_dx, image_name = y
        with torch.no_grad():
            output = self(x)
        return output, y_dx, image_name

    def setup(self, stage=None):
        metadata = pd.read_csv(self.metadata_file)
        train_set = metadata[metadata['split'] == 'train']
        self.train_set = MelanomaDataset(root_dir=self.data_dir,
                                                            metadata=train_set,
                                                            transform=self.train_transform)
        val_set = metadata[metadata['split'] == 'val']
        self.val_set = MelanomaDataset(root_dir=self.data_dir,
                                                          metadata=val_set,
                                                          transform=self.test_transform)
        test_set = metadata[metadata['split'] == 'test']
        self.test_set = MelanomaDataset(root_dir=self.data_dir,
                                                           metadata=test_set,
                                                           transform=self.test_transform)

    def train_dataloader(self):
        if self.weighted_sampling:
            y = self.train_set.metadata[self.labels].values.flatten().astype(int)
            counts = np.bincount(y)
            labels_weights = 1. / counts
            weights = labels_weights[y]
            sampler = WeightedRandomSampler(weights, num_samples=len(weights))
            return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=False, sampler=sampler, num_workers=16)
        else:
            return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=16)

    def val_dataloader(self):
        if False:# self.weighted_sampling:
            y = self.val_set.metadata[self.labels].values.flatten().astype(int)
            counts = np.bincount(y)
            labels_weights = 1. / counts
            weights = labels_weights[y]
            sampler = WeightedRandomSampler(weights, len(weights))
            return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False, sampler=sampler, num_workers=16)
        else:
            return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False, num_workers=16)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False, num_workers=16)

    def predict_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False, num_workers=16)

# Training

### ResNet18

In [ ]:
BASE_MODEL = 'resnet18'

CHECKPOINTS_DIR = os.path.join(BASE_DIR, f"binary_{BASE_MODEL}")
FINAL_CHECKPOINTS = os.path.join(CHECKPOINTS_DIR, "final_checkpoints.ckpt")
LOG_DIR = os.path.join(CHECKPOINTS_DIR, "logs")

In [ ]:
torch.cuda.empty_cache()

# Define model object and trainer.
model = MelanomaClassifier(backbone_model=BASE_MODEL, data_dir=IMG_DIR,
                           weighted_sampling=WEIGHTED_SAMPLING, batch_size=BATCH_SIZE,
                           learning_rate=LEARNING_RATE)
model.to("cuda")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 192MB/s]


MelanomaClassifier(
  (loss): BCELoss()
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, mo

In [ ]:
logger = TensorBoardLogger(save_dir=CHECKPOINTS_DIR)

In [ ]:
trainer = Trainer(
    max_epochs=NUM_EPOCHS,
    devices=1,
    accelerator="gpu",
    deterministic=True,
    enable_progress_bar=True,
    default_root_dir=CHECKPOINTS_DIR,
    logger=logger,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | loss        | BCELoss           | 0      | train
1 | base_model  | ResNet            | 11.2 M | train
2 | sigm        | Sigmoid           | 0      | train
3 | accuracy    | BinaryAccuracy    | 0      | train
4 | auroc       | BinaryAUROC       | 0      | train
5 | sensitivity | BinaryRecall      | 0      | train
6 | specificity | BinarySpecificity | 0      |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
trainer.save_checkpoint(FINAL_CHECKPOINTS)
print(f"Final checkpoints saved to {FINAL_CHECKPOINTS}")

Final checkpoints saved to /content/drive/MyDrive/DATABase/binary_resnet18/final_checkpoints.ckpt


### ResNet34

In [ ]:
BASE_MODEL = 'resnet34'

CHECKPOINTS_DIR_34 = os.path.join(BASE_DIR, f"binary_{BASE_MODEL}")
FINAL_CHECKPOINTS_34 = os.path.join(CHECKPOINTS_DIR_34, "final_checkpoints.ckpt")
LOG_DIR_34 = os.path.join(CHECKPOINTS_DIR_34, "logs")

torch.cuda.empty_cache()

# Define model object and trainer.
model_34 = MelanomaClassifier(backbone_model=BASE_MODEL, data_dir=IMG_DIR,
                           weighted_sampling=WEIGHTED_SAMPLING, batch_size=BATCH_SIZE,
                           learning_rate=LEARNING_RATE)
model_34.to("cuda")


logger = TensorBoardLogger(save_dir=LOG_DIR_34)

trainer_34 = Trainer(
    max_epochs=NUM_EPOCHS,
    devices=1,
    accelerator="gpu",
    deterministic=True,
    enable_progress_bar=True,
    default_root_dir=CHECKPOINTS_DIR_34,
    logger=logger,
)

trainer_34.fit(model_34)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 214MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | loss        | BCELoss           | 0      | train
1 | base_model  | ResNet            | 21.3 M | train
2 | sigm        | Sigmoid           | 0      | train
3 | accuracy    | BinaryAccuracy    | 0      | train
4 | auroc       | BinaryAUROC       | 0      | train
5 | sensitivity | BinaryRecall      | 0      | train
6 | speci

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
trainer_34.save_checkpoint(FINAL_CHECKPOINTS_34)
print(f"Final checkpoints saved to {FINAL_CHECKPOINTS_34}")

Final checkpoints saved to /content/drive/MyDrive/DATABase/binary_resnet34/final_checkpoints.ckpt


### ResNet50

In [ ]:
BASE_MODEL = 'resnet50'

CHECKPOINTS_DIR_50 = os.path.join(BASE_DIR, f"binary_{BASE_MODEL}")
FINAL_CHECKPOINTS_50 = os.path.join(CHECKPOINTS_DIR_50, "final_checkpoints.ckpt")
LOG_DIR_50 = os.path.join(CHECKPOINTS_DIR_50, "logs")

torch.cuda.empty_cache()

# Define model object and trainer.
model_50 = MelanomaClassifier(backbone_model=BASE_MODEL, data_dir=IMG_DIR,
                           weighted_sampling=WEIGHTED_SAMPLING, batch_size=BATCH_SIZE,
                           learning_rate=LEARNING_RATE)
model_50.to("cuda")


logger = TensorBoardLogger(save_dir=LOG_DIR_50)

trainer_50 = Trainer(
    max_epochs=NUM_EPOCHS,
    devices=1,
    accelerator="gpu",
    deterministic=True,
    enable_progress_bar=True,
    default_root_dir=CHECKPOINTS_DIR_50,
    logger=logger,
)

trainer_50.fit(model_50)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | loss        | BCELoss           | 0      | train
1 | base_model  | ResNet            | 23.5 M | train
2 | sigm        | Sigmoid           | 0      | train
3 | accuracy    | BinaryAccuracy    | 0      | train
4 | auroc       | BinaryAUROC       | 0      | train
5 | sensitivity | BinaryRecall      | 0      | train
6 | speci

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
trainer_50.save_checkpoint(FINAL_CHECKPOINTS_50)
print(f"Final checkpoints saved to {FINAL_CHECKPOINTS_50}")

Final checkpoints saved to /content/drive/MyDrive/DATABase/binary_resnet50/final_checkpoints.ckpt


### ResNet101

In [ ]:
BASE_MODEL = 'resnet101'

CHECKPOINTS_DIR_101 = os.path.join(BASE_DIR, f"binary_{BASE_MODEL}")
FINAL_CHECKPOINTS_101 = os.path.join(CHECKPOINTS_DIR_101, "final_checkpoints.ckpt")
LOG_DIR_101 = os.path.join(CHECKPOINTS_DIR_101, "logs")

torch.cuda.empty_cache()

# Define model object and trainer.
model_101 = MelanomaClassifier(backbone_model=BASE_MODEL, data_dir=IMG_DIR,
                           weighted_sampling=WEIGHTED_SAMPLING, batch_size=BATCH_SIZE,
                           learning_rate=LEARNING_RATE)
model_101.to("cuda")


logger = TensorBoardLogger(save_dir=LOG_DIR_101)

trainer_101 = Trainer(
    max_epochs=NUM_EPOCHS,
    devices=1,
    accelerator="gpu",
    deterministic=True,
    enable_progress_bar=True,
    default_root_dir=CHECKPOINTS_DIR_101,
    logger=logger,
)

trainer_101.fit(model_101)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 207MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type              | Params | Mode 
----------------------------------------------------------
0 | loss        | BCELoss           | 0      | train
1 | base_model  | ResNet            | 42.5 M | train
2 | sigm        | Sigmoid           | 0      | train
3 | accuracy    | BinaryAccuracy    | 0      | train
4 | auroc       | BinaryAUROC       | 0      | train
5 | sensitivity | BinaryRecall      | 0      | train
6 | speci

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=25` reached.


In [ ]:
trainer_101.save_checkpoint(FINAL_CHECKPOINTS_101)
print(f"Final checkpoints saved to {FINAL_CHECKPOINTS_101}")

Final checkpoints saved to /content/drive/MyDrive/DATABase/binary_resnet101/final_checkpoints.ckpt


# Testing

### Loading the models

In [ ]:
base_models = ['ResNet18', 'ResNet34', 'ResNet50', 'ResNet101']
trained_models = []
trainers = []

for base_model in base_models:
    checkpoints_path = os.path.join(BASE_DIR, f"binary_{base_model.lower()}/final_checkpoints.ckpt")
    trained_models.append(MelanomaClassifier.load_from_checkpoint(checkpoints_path,
                                                          map_location="cuda",
                                                          backbone_model=base_model.lower()))
    trained_models[-1].to("cuda")
    trained_models[-1].setup()
    trainers.append(Trainer(devices=1, accelerator="gpu", enable_progress_bar=True))
    print(f"Model with {base_model} backbone loaded successfully!")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Model with ResNet18 backbone loaded successfully!


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Model with ResNet34 backbone loaded successfully!


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Model with ResNet50 backbone loaded successfully!


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Model with ResNet101 backbone loaded successfully!


### Prediction and evaluating

In [ ]:
def get_dx_predictions(trainer, model, split='test', threshold=0.5):
    """
    Stores predictions, scores, and true values in a DataFrame.
    """
    if split == 'test':
        predictions = trainer.predict(model, model.test_dataloader())
    else:
        predictions = trainer.predict(model, model.val_dataloader())

    dfs = []

    for preds in predictions:

        y_pred, y_true, image_name = preds
        df = pd.DataFrame(y_pred, columns=['score'])
        y_pred = torch.where(y_pred >= threshold, 1, 0)

        df['pred'] = y_pred
        df['true'] = y_true

        dfs.append(df)

    result = pd.concat(dfs, ignore_index=True).dropna()

    return result

In [ ]:
def find_optimal_cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = metrics.roc_curve(target, predicted)
    i = np.arange(len(tpr))
    roc = pd.DataFrame({'tf': pd.Series(tpr - (1 - fpr), index=i), 'threshold': pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf - 0).abs().argsort()[:1]]

    return roc_t['threshold'].item()

In [ ]:
def display_scores(result):
    true = 'true'
    score = 'score'
    pred = 'pred'
    print('=====')
    print('AUC:', metrics.roc_auc_score(result[true], result[score]))
    print('Balanced Acc:', metrics.balanced_accuracy_score(result[true], result[pred]))
    print('Sensitivity:', metrics.recall_score(result[true], result[pred]))
    print('Specificity:', metrics.recall_score(result[true], result[pred], pos_label=0))
    print('=====\n')

In [ ]:
def evaluate_model(trainer, model, model_name, val_results, thresholds, test_results):
    print('=================================')
    print(f'Model with {model_name} backbone')
    val_results[model_name] = get_dx_predictions(trainer, model, split='val', threshold=0.5)

    print('Val:')
    display_scores(val_results[model_name])
    thresholds[model_name] = find_optimal_cutoff(val_results[model_name]['true'], val_results[model_name]['score'])

    test_results[model_name] = get_dx_predictions(trainer, model, split='test', threshold=thresholds[model_name])
    print('Test:')
    display_scores(test_results[model_name])
    print('=================================\n\n')

In [ ]:
val_results, thresholds, test_results = dict(), dict(), dict()

In [ ]:
for i in range(len(base_models)):
    evaluate_model(trainers[i], trained_models[i], base_models[i], val_results, thresholds, test_results)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model with ResNet18 backbone


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val:
=====
AUC: 0.861127684333016
Balanced Acc: 0.7981169998942135
Sensitivity: 0.7159420289855073
Specificity: 0.8802919708029197
=====



...

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Val:
=====
AUC: 0.816303818893473
Balanced Acc: 0.7237067597588067
Sensitivity: 0.5043478260869565
Specificity: 0.9430656934306569
=====



Predicting: |          | 0/? [00:00<?, ?it/s]

Test:
=====
AUC: 0.8619756804997769
Balanced Acc: 0.7867302543507363
Sensitivity: 0.7361111111111112
Specificity: 0.8373493975903614
=====





In [ ]:
SAVE_DIR = os.path.join(BASE_DIR, "binary_classification_results")
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
with open(os.path.join(SAVE_DIR, "validation_results.pkl"), "wb") as f:
    pickle.dump(val_results, f)
with open(os.path.join(SAVE_DIR, "thresholds.pkl"), "wb") as f:
    pickle.dump(thresholds, f)
with open(os.path.join(SAVE_DIR, "test_results.pkl"), "wb") as f:
    pickle.dump(test_results, f)